In [1]:
import pandas as pd
import os
import glob
import numpy as np


In [3]:
# 데이터프레임 이름은 df로 설정
# 데이터 모든컬럼 헤더데이터 통합
df = pd.read_csv("../../data/통합데이터/Final_merged_all_data.csv")

In [8]:
cols = df.columns
# with open('columns_all.txt', 'w', encoding='utf-8') as f:
#     f.write('\n'.join(cols))
len(cols)

906

In [11]:
import pandas as pd
import numpy as np

# df = pd.read_csv("Final_merged_all_data.csv") 

# -------------------------------------------------------
# 1. 소비 추세 (Trend) - R6M과 R3M을 활용한 구간 비교
# -------------------------------------------------------
# R3M: 최근 3개월 합계 (M0 + M-1 + M-2)
# R6M: 최근 6개월 합계 (M0 ... M-5)
# 따라서 (R6M - R3M)은 '4개월 전 ~ 6개월 전'의 합계가 됩니다.

col_r3m = '이용금액_신판_R3M'
col_r6m = '이용금액_신판_R6M'

if col_r3m in df.columns and col_r6m in df.columns:
    # 0 결측치 처리
    df[col_r3m] = df[col_r3m].fillna(0)
    df[col_r6m] = df[col_r6m].fillna(0)
    
    # 1. 최근 3개월 월평균 (Recent Average)
    df['Avg_Recent_3M'] = df[col_r3m] / 3
    
    # 2. 과거 3개월(4~6개월 전) 월평균 (Past Average)
    # R6M이 R3M보다 작을 순 없으므로 max(0, ...) 처리
    df['Sum_Past_3M'] = (df[col_r6m] - df[col_r3m]).clip(lower=0)
    df['Avg_Past_3M'] = df['Sum_Past_3M'] / 3
    
    # 3. 변동률 계산 (Trend)
    # (최근 - 과거) / 과거
    # 과거 데이터가 0이면(신규 등), 변동률 0으로 처리
    df['Trend_Change_Rate'] = np.where(df['Avg_Past_3M'] > 0,
                                       (df['Avg_Recent_3M'] - df['Avg_Past_3M']) / df['Avg_Past_3M'],
                                       0)
else:
    print("⚠️ R3M, R6M 컬럼이 없습니다. Trend를 0으로 설정합니다.")
    df['Trend_Change_Rate'] = 0


# -------------------------------------------------------
# 2. 활동성 비율 (Ratio) - B0M vs R3M
# -------------------------------------------------------
col_current = '이용금액_신판_B0M' # 이건 있다고 가정 (없으면 이용금액_신용_B0M 사용)

if col_current in df.columns and 'Avg_Recent_3M' in df.columns:
    df['Activity_Ratio'] = np.where(df['Avg_Recent_3M'] > 0, 
                                    df[col_current] / df['Avg_Recent_3M'], 
                                    1.0)
else:
    df['Activity_Ratio'] = 1.0


# -------------------------------------------------------
# 3. 위험 점수 (Risk Score) - 기존 동일
# -------------------------------------------------------
df['Risk_Score'] = 0

if '연체잔액_B0M' in df.columns:
    df['Risk_Score'] += np.where(df['연체잔액_B0M'] > 0, 50, 0)
if '잔액_현금서비스_B0M' in df.columns:
    df['Risk_Score'] += np.where(df['잔액_현금서비스_B0M'] > 1000000, 20, 0)
if '승인거절건수_B0M' in df.columns:
    df['Risk_Score'] += np.where(df['승인거절건수_B0M'] > 0, 10, 0)


# -------------------------------------------------------
# 4. 최종 타겟 생성 (Churn_Probabilistic)
# -------------------------------------------------------
# 조건 수정:
# 1. Trend_Change_Rate <= -0.3 (과거 대비 최근 30% 이상 감소)
# 2. Activity_Ratio <= 0.3 (평소 대비 이번 달 30% 이하 사용)
# 3. Risk_Score >= 50 (연체 등)

df['Churn_Probabilistic'] = np.where(
    (df['Trend_Change_Rate'] <= -0.3) | 
    (df['Activity_Ratio'] <= 0.3) | 
    (df['Risk_Score'] >= 50),
    1, 0
)

print("\n[최종 결과 확인 - R6M 활용 버전]")
print(df[['발급회원번호', 'Avg_Past_3M', 'Avg_Recent_3M', 'Trend_Change_Rate', 'Churn_Probabilistic']].head(10))
print("\n이탈 예측 비율:")
print(df['Churn_Probabilistic'].value_counts(normalize=True))


[최종 결과 확인 - R6M 활용 버전]
  발급회원번호   Avg_Past_3M  Avg_Recent_3M  Trend_Change_Rate  Churn_Probabilistic
0  SYN_0  6.332233e+04   1.608897e+05           1.540804                    0
1  SYN_1  1.711770e+05   4.763600e+05           1.782850                    0
2  SYN_2  5.277267e+04   8.106700e+04           0.536155                    1
3  SYN_3  0.000000e+00   1.303867e+04           0.000000                    0
4  SYN_4  4.289333e+04  -4.289333e+04          -2.000000                    1
5  SYN_5  3.383833e+04  -3.383833e+04          -2.000000                    1
6  SYN_6  2.110567e+04  -2.110567e+04          -2.000000                    1
7  SYN_7  7.807867e+04  -5.731600e+04          -1.734080                    1
8  SYN_8  1.568185e+06   1.532504e+06          -0.022753                    0
9  SYN_9  2.245343e+05   1.044223e+05          -0.534938                    1

이탈 예측 비율:
Churn_Probabilistic
0    0.724633
1    0.275367
Name: proportion, dtype: float64
